In [ ]:
 # This Python 3 environment comes with many helpful analytics libraries installed
 # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
 # For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

 =============================================================================
 STEP 1: ENVIRONMENT SETUP AND LIBRARY INSTALLATION
 =============================================================================

 In this step, we install all the necessary Python packages. Kaggle environments
 are ephemeral, so these installations are required for each session. We use
 quiet flags (-q) to keep the output clean.

 Key Libraries:
 - langchain: The core framework for building LLM applications.
 - transformers: Provides access to Hugging Face models and pipelines.
 - faiss-cpu: A library for efficient similarity search (our vector store).
 - sentence-transformers: Used for generating high-quality text embeddings.
 - pypdf: A robust library for extracting text from PDF documents.
 - accelerate: Required for efficiently loading and running large models on GPUs.

 =============================================================================

In [ ]:
pip uninstall sentence-transformers--y

In [2]:
print("STEP 1: Installing required libraries...")
# This consolidated and upgraded installation command helps pip resolve
# dependencies more effectively in the complex Kaggle environment.
!pip install -q --upgrade --no-cache-dir \
    "transformers>=4.41.0" \
    "accelerate>=0.30.0" \
    torch \
    "langchain>=0.2.0" \
    "langchain-community[pdf]>=0.2.0" \
    faiss-cpu \
    sentence-transformers \
    pypdf

print("Library installation complete.\n")

STEP 1: Installing required libraries...
Library installation complete.



 =============================================================================
 STEP 2: LOAD THE PRE-BUILT VECTOR INDEX
 =============================================================================

 Instead of processing PDFs, we directly load the saved FAISS index.
 This is significantly faster.

=============================================================================

In [3]:
import os
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

print("STEP 2: Loading the pre-built vector index...")

# Define the path to the saved index
index_path = "/kaggle/input/gate-dsai-llm/my_vector_index"

# Check if the index exists before trying to load
if not os.path.exists(index_path):
    raise FileNotFoundError(
        f"Index not found at '{index_path}'. "
        "Please run the 'build_index.ipynb' notebook first to create it."
    )

# Initialize the same embedding model used to create the index
embedding_model_name = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name)

# Load the vector store from disk
vector_store = FAISS.load_local(index_path, embeddings, allow_dangerous_deserialization=True)

print("Vector index loaded successfully.\n")

STEP 2: Loading the pre-built vector index...


ImportError: Could not import sentence_transformers python package. Please install it with `pip install sentence-transformers`.

 =============================================================================
 STEP 3: LARGE LANGUAGE MODEL (LLM) INITIALIZATION
 =============================================================================

 We now load our generative AI model. We are using Microsoft's Phi-3-mini,
 a state-of-the-art small language model known for high-quality reasoning
 and generation. We use the 'transformers' library to create a pipeline,
 which is then wrapped by LangChain for seamless integration.

 `device_map="auto"` intelligently utilizes the available Kaggle GPU.

=============================================================================

In [ ]:
from langchain_community.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

print("STEP 3: Initializing the Large Language Model (Phi-3-mini)...")
print("(This may take a few minutes as the model is loaded into the GPU)")

# Define the model ID from Hugging Face
model_id = "microsoft/Phi-3-mini-4k-instruct"

# Load the model and its tokenizer.
# `trust_remote_code=True` is required for this model architecture.
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    torch_dtype="auto",
    device_map="auto" # This automatically uses the GPU
)

# Create a text-generation pipeline.
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,  # Max length of the generated answer
    return_full_text=False # We only want the generated response
)

# Wrap the pipeline in a LangChain-compatible object.
llm = HuggingFacePipeline(pipeline=pipe)

print("LLM initialized successfully.\n")

 =============================================================================
 STEP 4: CREATING THE QUESTION-ANSWERING (QA) CHAIN
 =============================================================================

 This step ties everything together. We create a `RetrievalQA` chain, which
 orchestrates the entire RAG process. We use the "refine" chain type, which
 is robust for detailed questions. It iterates through the retrieved documents,
 progressively building and refining the answer, which helps generate more
 comprehensive and accurate responses.

 =============================================================================

In [ ]:
from langchain.chains import RetrievalQA

print("STEP 4: Creating the RetrievalQA chain...")

# The "refine" chain type is ideal for generating detailed answers from
# multiple document sources.
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="refine",
    retriever=vector_store.as_retriever(),
    return_source_documents=True,
)

print("QA chain is ready.\n")


 =============================================================================
 STEP 5: INTERACTIVE CHATBOT INTERFACE
 =============================================================================

 This final block runs a simple command-line interface, allowing you to
 interact with your custom GATE DSAI study bot.

 =============================================================================

In [ ]:
print("--- GATE DSAI Study Bot Initialized ---")
print("You can now ask questions about your documents.")
print("Type 'quit' or 'exit' to end the session.")

while True:
    try:
        query = input("\nYour Question: ")
        if query.lower() in ["quit", "exit"]:
            print("Session ended. Goodbye!")
            break
        if not query.strip():
            continue

        # Invoke the QA chain with the user's query
        result = qa_chain.invoke({"query": query})

        # Display the results
        print("\nAnswer:")
        print(result["result"])
        print("\nSources Used:")
        # Display the source documents for verification
        for doc in result["source_documents"]:
            print(f"  - {doc.metadata.get('source', 'Unknown source')}")

    except Exception as e:
        print(f"\nAn error occurred: {e}")